<a href="https://colab.research.google.com/github/phongvu009/micro_grad/blob/main/Simple_Training_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math


In [ ]:
random.seed(8)

In [ ]:
class Value :
  def __init__(self, data, _children=(), _op='', label = ''):
    self.data = data
    self._prev = _children
    self._op = _op
    self.label = label
    self._backward = lambda : None
    self.grad = 0.0

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self,other):
    #deal with integer : wrap integer into Value class
    other = other if isinstance(other,Value) else Value(other)
    out = Value(self.data + other.data, (self,other),  '+')
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    return out

  def __neg__(self): # -self
    return self * -1

  def __sub__(self,other): # self - other
    return self + (-other)

  def __rsub__(self,other): #other - self
    return other + (-self)

  def __radd__(self,other):
    return self + other
  #deal with integer * Value_object
  def __rmul__(self,other):
    return self * other

  def __mul__(self,other):
    other = other if isinstance(other,Value) else Value(other)
    out = Value(self.data * other.data, (self,other), '*')
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out

  #do division
  def __truediv__(self,other): #self/other
    return self * other**-1

  def __pow__(self,other):
    assert isinstance(other,(int,float)) # only accept int or float
    out = Value( self.data ** other, (self,) , f'**{other}')
    def _backward():
      self.grad += (other * self.data ** (other -1)) * out.grad
    out._backward = _backward
    return out

  def tanh(self):
    x = self.data
    t = (math.exp(2*x)-1) / (math.exp(2*x)+1)
    out = Value(t, (self,), 'tanh')
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    return out

  def exp(self):
    x = self.data
    out = Value( math.exp(x), (self,), 'exp')
    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward
    return out

  def backward(self):
    topo = []
    visited = set()
    def build_topo(current_node):
      if current_node not in visited:
        visited.add(current_node)
        for child in current_node._prev:
          build_topo(child)
        topo.append(current_node)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

class Neuron:
  '''
  To create a basic form: inputs -> 1 output
  '''
  def __init__(self,nin):
    #initialize random weights and bias w.r.t inputs
    self.w = [ Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))

  def __call__(self,x):
    #zip() help to pair random weights with inputs
    act = sum([wi*xi for wi,xi in zip(self.w,x) ])
    out = act.tanh()
    return out

  def parameters(self):
    return self.w + [self.b]

class Layer:
  def __init__(self,nin,nout):
    self.neurons = [ Neuron(nin) for _ in range(nout) ]

  def __call__(self,x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def parameters(self):
    return [ p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    #to initilize weights and bias for more layer
    self.layers = [ Layer(sz[i],sz[i+1]) for i in range(len(nouts)) ]

  def __call__(self,x):
    for layer in self.layers:
      x = layer(x)
    return x

  def parameters(self):
    #help to get param for multi-layer
    return [ p for layer in self.layers for p in layer.parameters()]



In [ ]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys =[1.0,-1.0,-1.0,1.0]

## Simple Training Loop

In [ ]:
n = MLP(3,[4,4,1])

In [ ]:
for k in range(20):
  #forward
  pred = [n(x) for x in xs]
  loss = sum( [(ys-pred)**2 for ys,pred in zip(ys,pred)])

  #need to zero grad after backward : new grad after new update
  # otherwise it will acc prevous grad
  for p in n.parameters():
    p.grad =0

  #backward
  loss.backward()

  #update weights
  for p in n.parameters():
    p.data += -0.05 *  p.grad

  print(f' at {k} -- the loss is : {loss.data} ')



 at 0 -- the loss is : 6.838684685676855 
 at 1 -- the loss is : 1.1781828422544471 
 at 2 -- the loss is : 0.5752382700337307 
 at 3 -- the loss is : 0.35630805750868577 
 at 4 -- the loss is : 0.24677779967573352 
 at 5 -- the loss is : 0.184369459913283 
 at 6 -- the loss is : 0.14519276471431294 
 at 7 -- the loss is : 0.11874483321217436 
 at 8 -- the loss is : 0.09988054007997223 
 at 9 -- the loss is : 0.0858421421938274 
 at 10 -- the loss is : 0.07503947057738908 
 at 11 -- the loss is : 0.06649958568193502 
 at 12 -- the loss is : 0.059597562477572044 
 at 13 -- the loss is : 0.05391550020742113 
 at 14 -- the loss is : 0.04916431555004119 
 at 15 -- the loss is : 0.04513819497728726 
 at 16 -- the loss is : 0.04168693924696691 
 at 17 -- the loss is : 0.038698557207396414 
 at 18 -- the loss is : 0.03608796257580577 
 at 19 -- the loss is : 0.03378942987645646 


In [ ]:
pred

[Value(data=0.9430602545179209),
 Value(data=-0.9260205026444556),
 Value(data=-0.8637048083926964),
 Value(data=0.919390137224767)]